In [1]:
#!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import torch
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, PreTrainedTokenizerFast, BartTokenizer
import pandas as pd
import torch
from torch.utils.data import random_split, Dataset
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
import numpy as np

In [2]:
tokenizer = get_kobart_tokenizer()
model = AutoModelForSequenceClassification.from_pretrained(get_pytorch_kobart_model(), num_labels=7)

using cached model
using cached model


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at /root/kobart/kobart_from_pretrained and are newly initialized: ['classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.dense.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
class Newsdatasets(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [4]:
class Testsets(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [5]:
train_data = pd.read_csv('datasets/train_data.csv')
test_data = pd.read_csv('datasets/test_data.csv')
text_input = [item + tokenizer.eos_token for item in train_data.title.tolist()]
train_encodings = tokenizer(text_input, max_length=50, truncation=True, padding=True, return_tensors='pt')
del train_encodings['token_type_ids']

text_input = [item + tokenizer.eos_token for item in test_data.title.tolist()]
val_encodings = tokenizer(text_input, max_length=50, truncation=True, padding=True, return_tensors='pt', )
del val_encodings['token_type_ids']

In [6]:
train_datasets = Newsdatasets(train_encodings, torch.tensor(train_data.topic_idx.to_list()))
train_size = int(len(train_datasets)*0.7)
train_dataset, val_dataset = random_split(train_datasets, [train_size, len(train_datasets)-train_size])
testsets = Testsets(val_encodings)

In [7]:
training_args = TrainingArguments(
    output_dir = "dacon_test",
    per_device_train_batch_size = 512,
    per_device_eval_batch_size = 512,
    learning_rate=3e-05
    )

In [8]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset
)

In [9]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Step,Training Loss


TrainOutput(global_step=189, training_loss=0.5756713624984499, metrics={'train_runtime': 166.5335, 'train_samples_per_second': 1.135, 'total_flos': 2076118533775422, 'epoch': 3.0})

In [10]:
results = trainer.predict(
    test_dataset = testsets)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [11]:
submission=pd.read_csv('datasets/sample_submission.csv')
submission['topic_idx'] = np.argmax(results.predictions[0], axis=1)
submission.columns = ['index', 'topic_idx']
submission.to_csv('kobart1.csv', encoding='utf-8', index=False)